In [15]:
from keras.models import Sequential#to create a models with inputs and outputs 
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2#old library of openCV
import os#comes with standard library of the python
import numpy as np
import sys
from numpy import genfromtxt#to create arrays for the tabular data
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=sys.maxsize)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
#loading the pretrained model
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

In [17]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    #this is the loss function
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    # Compute the (encoding) distance between the anchor and the positive
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis=None)
    # Compute the (encoding) distance between the anchor and the negative
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis=None)
    # subtract the two previous distances and add alpha.
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    # Take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0.0))
    
    return loss

In [18]:
#here we have  create the triplet loss as the loss function 
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

In [19]:
database = {}
database["Kavindu"] = img_to_encoding("images/kavindu.jpg", FRmodel)
#database["Deshan"] = img_to_encoding("images/deshan.jpg", FRmodel)

In [20]:
def who_is_it(image_path, database, model):
    
    # Compute the target "encoding" for the image. Use img_to_encoding() see example above.
    encoding = img_to_encoding(image_path, model)
    # Initialize "min_dist" to a large value, say 100 
    min_dist = 100
    
    # Loop over the database dictionary's names and encodings.
    for (name, db_enc) in database.items():#db_enc is database encoding
        # Compute L2 distance between the target "encoding" and the current "emb" from the database
        dist = np.linalg.norm(db_enc - encoding)
        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name
        if dist<min_dist:
            min_dist = dist
            identity = name
    
    if min_dist > 0.7:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))
        
    return min_dist, identity

In [21]:
who_is_it("images/camera_6.jpg", database, FRmodel)

it's Kavindu, the distance is 0.5951577


(0.5951577, 'Kavindu')